#PREPROCESSING

In [14]:
# 01_preprocess.py
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import re
from pathlib import Path
import logging

# Configuración de rutas
INPUT_PATH = Path(r"C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\data\secuencias.fasta.txt")
RESULTS_PATH = Path(r"C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\results\2025-03-30_FG\identificacion_patogeno")

def setup_logging():
    """Configura el sistema de logging"""
    RESULTS_PATH.mkdir(parents=True, exist_ok=True)
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(RESULTS_PATH / "preprocess.log"),
            logging.StreamHandler()
        ]
    )

def clean_sequences(input_path: Path, output_path: Path) -> None:
    """
    Limpia y filtra secuencias FASTA:
    - Elimina caracteres no estándar
    - Convierte a mayúsculas
    - Filtra secuencias demasiado cortas
    - Separa secuencias concatenadas con 'xxxxx'
    """
    clean_seqs = []
    
    for record in SeqIO.parse(input_path, "fasta"):
        # Procesar cada secuencia
        seq = str(record.seq).upper()
        
        # Separar secuencias concatenadas con 'xxxxx'
        sub_seqs = re.split(r'[XxNn]{4,}', seq)
        
        for i, sub_seq in enumerate(sub_seqs):
            # Limpiar caracteres no estándar
            clean_seq = re.sub(r'[^ATCG]', '', sub_seq)
            
            if len(clean_seq) >= 100:  # Filtro por longitud mínima
                # Crear nuevo registro con ID único
                new_id = f"{record.id}_part{i+1}" if len(sub_seqs) > 1 else record.id
                clean_record = SeqRecord(
                    Seq(clean_seq),
                    id=new_id,
                    description=f"Cleaned version of {record.id}, length={len(clean_seq)}"
                )
                clean_seqs.append(clean_record)
    
    # Guardar resultados en archivo FASTA normal (sin comprimir)
    with open(output_path, 'w') as f:
        SeqIO.write(clean_seqs, f, "fasta")
    
    logging.info(f"Procesadas {len(clean_seqs)} secuencias. Guardadas en: {output_path}")
    logging.info(f"Puedes abrir el archivo en un editor de texto: {output_path}")

if __name__ == "__main__":
    setup_logging()
    try:
        output_file = RESULTS_PATH / "cleaned_sequences.fasta"
        clean_sequences(INPUT_PATH, output_file)
    except Exception as e:
        logging.error(f"Error en preprocesamiento: {e}", exc_info=True)

2025-04-05 16:07:00,441 - INFO - Procesadas 24 secuencias. Guardadas en: C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\results\2025-03-30_FG\identificacion_patogeno\cleaned_sequences.fasta
2025-04-05 16:07:00,445 - INFO - Puedes abrir el archivo en un editor de texto: C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\results\2025-03-30_FG\identificacion_patogeno\cleaned_sequences.fasta


#BLAST

In [38]:
# 02_blast.py
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO
import pandas as pd
from pathlib import Path
import logging
import re
from typing import Optional

# Configuración básica
RESULTS_PATH = Path(r"C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\results\2025-03-30_FG\identificacion_patogeno")
DEFAULT_PARAMS = {
    "program": "blastn",
    "database": "refseq_viral",
    "e_value": 1e-10,
    "hitlist_size": 50,
    "word_size": 11,
    "gapcosts": "5 2"  # Solo parámetros soportados por qblast
}

def setup_logging():
    """Configura el sistema de logging"""
    RESULTS_PATH.mkdir(parents=True, exist_ok=True)
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(RESULTS_PATH / "blast_analysis.log"),
            logging.StreamHandler()
        ]
    )

def run_remote_blast(
    query_file: Path,
    program: str = DEFAULT_PARAMS["program"],
    database: str = DEFAULT_PARAMS["database"],
    e_value: float = DEFAULT_PARAMS["e_value"],
    hitlist_size: int = DEFAULT_PARAMS["hitlist_size"],
    word_size: int = DEFAULT_PARAMS["word_size"],
    gapcosts: str = DEFAULT_PARAMS["gapcosts"],
) -> Path:
    """
    Ejecuta BLAST remoto con parámetros personalizables
    
    Args:
        query_file: Archivo FASTA con secuencias
        program: blastn, blastp, etc.
        database: Base de datos a usar
        e_value: Umbral de significancia
        hitlist_size: Número máximo de resultados
        word_size: Tamaño de palabra (7-15 para blastn)
        gapcosts: Costos de gaps ("abrir extender")
        
    Returns:
        Ruta al archivo XML con resultados
    """
    output_file = RESULTS_PATH / "blast_results.xml"
    
    records = list(SeqIO.parse(query_file, "fasta"))
    if not records:
        raise ValueError("No se encontraron secuencias en el archivo de consulta")
    
    logging.info(f"Iniciando BLAST remoto con {len(records)} secuencias")
    logging.info(f"Parámetros: word_size={word_size}, gapcosts={gapcosts}, evalue={e_value}")

    with open(query_file) as f:
        fasta_data = f.read()
    
    try:
        # Solo parámetros soportados por NCBIWWW.qblast()
        blast_params = {
            "program": program,
            "database": database,
            "sequence": fasta_data,
            "expect": e_value,
            "hitlist_size": hitlist_size,
            "format_type": "XML",
            "word_size": word_size,
            "gapcosts": gapcosts,
        }
        
        handle = NCBIWWW.qblast(**blast_params)
        
        with open(output_file, "w") as f:
            blast_results = handle.read()
            f.write(blast_results)
        
        logging.info(f"Resultados guardados en: {output_file}")
        return output_file
        
    except Exception as e:
        logging.error(f"Error en BLAST remoto: {e}")
        raise

def parse_blast_results(xml_path: Path) -> pd.DataFrame:
    """Procesa resultados XML de BLAST"""
    hits = []
    
    try:
        with open(xml_path, 'r') as f:
            blast_records = NCBIXML.parse(f)
            
            for record in blast_records:
                if not record.alignments:
                    continue
                    
                for align in record.alignments:
                    for hsp in align.hsps:
                        hit_def = align.hit_def
                        organism = re.search(r'\[(.*?)\]', hit_def)
                        organism = organism.group(1) if organism else "Desconocido"
                        
                        hits.append({
                            "query_id": record.query,
                            "query_length": record.query_length,
                            "hit_accession": align.accession,
                            "hit_definition": hit_def,
                            "organism": organism,
                            "evalue": hsp.expect,
                            "bit_score": hsp.bits,
                            "alignment_length": hsp.align_length,
                            "percent_identity": hsp.identities / hsp.align_length * 100,
                            "query_start": hsp.query_start,
                            "query_end": hsp.query_end,
                            "hit_start": hsp.sbjct_start,
                            "hit_end": hsp.sbjct_end,
                            "alignment": f"{hsp.query[0:50]}..." if len(hsp.query) > 50 else hsp.query
                        })
    except Exception as e:
        logging.error(f"Error al parsear resultados: {str(e)}")
        raise
    
    df = pd.DataFrame(hits)
    
    if not df.empty:
        df = df.sort_values(["evalue", "bit_score"], ascending=[True, False])
        df = df.drop_duplicates(subset=["query_id", "hit_accession"], keep="first")
    
    return df

def save_results(df: pd.DataFrame, params: dict):
    """Guarda los resultados en archivos CSV"""
    csv_path = RESULTS_PATH / "blast_results_detailed.csv"
    df.to_csv(csv_path, index=False)
    logging.info(f"Resultados detallados guardados en: {csv_path}")
    
    if not df.empty:
        summary = df.groupby("organism").agg({
            "evalue": "min",
            "bit_score": "max",
            "percent_identity": "mean",
            "query_id": "count",
            "hit_accession": lambda x: ", ".join(sorted(set(x)))
        }).sort_values(["evalue", "bit_score"], ascending=[True, False])
        
        summary_path = RESULTS_PATH / "blast_results_summary.csv"
        summary.to_csv(summary_path)
        
        params_path = RESULTS_PATH / "blast_parameters.txt"
        with open(params_path, 'w') as f:
            for key, value in params.items():
                f.write(f"{key}: {value}\n")
        
        logging.info(f"Resumen de resultados guardado en: {summary_path}")
        logging.info("\nMejores hits encontrados:\n" + str(summary.head(10)))
    else:
        logging.warning("No se encontraron hits significativos")

if __name__ == "__main__":
    setup_logging()
    
    try:
        query_seq = Path(r"C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\results\2025-03-30_FG\identificacion_patogeno\cleaned_sequences.fasta")
        
        if not query_seq.exists():
            raise FileNotFoundError(f"No se encontró el archivo de consulta: {query_seq}")
        
        # Parámetros personalizados (solo los soportados por qblast)
        custom_params = {
            "program": "blastn",
            "database": "env_nt",
            "e_value": 1e-3,
            "hitlist_size": 50,
            "word_size": 7,  # Ajustable entre 7-15
            "gapcosts": "5 2"  # Costos de gaps
        }
        
        blast_results = run_remote_blast(query_seq, **custom_params)
        df = parse_blast_results(blast_results)
        save_results(df, custom_params)
        
    except Exception as e:
        logging.error(f"Error inesperado: {str(e)}", exc_info=True)


        # Añade esto después de obtener los resultados vacíos
if df.empty:
    logging.warning(r"""
    No se encontraron hits significativos. Recomendaciones:
    1. Aumentar el e_value 
    2. Reducir el word_size 
    3. Cambiar la base de datos (Ver C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\Bases_de_datos_blast.xlsx )
    4. Verificar la calidad de las secuencias de entrada
    5. Considerar usar blastx si son secuencias codificantes
    """)
    
    # Generar archivo con recomendaciones
    with open(RESULTS_PATH / "recommendations.txt", "w") as f:
        f.write("Recomendaciones para mejorar los resultados:\n")
        f.write(f"1. Aumentar e_value: Actual {custom_params['e_value']} -> Prueba con 1e-3\n")
        f.write(f"2. Reducir word_size: Actual {custom_params['word_size']} -> Prueba con 7\n")
        f.write("3. Cambiar database: Actual 'refseq_viral' -> Prueba con 'nr' o 'nt'\n")
        f.write("4. Verificar secuencias de entrada (contaminación, calidad)\n")
        f.write("5. Para secuencias codificantes, considerar usar blastx\n")

2025-04-05 19:36:28,845 - INFO - Iniciando BLAST remoto con 24 secuencias
2025-04-05 19:36:28,848 - INFO - Parámetros: word_size=7, gapcosts=5 2, evalue=0.001


c:\python_env\proyecto_troncal2\Lib\site-packages\Bio\Blast\NCBIWWW.py:281: BiopythonWarning: BLAST request Z3C3D4JM013 is taking longer than 10 minutes, consider re-issuing it
  warnings.warn(
2025-04-05 19:47:30,869 - INFO - Resultados guardados en: C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\results\2025-03-30_FG\identificacion_patogeno\blast_results.xml
2025-04-05 19:47:30,902 - INFO - Resultados detallados guardados en: C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\results\2025-03-30_FG\identificacion_patogeno\blast_results_detailed.csv
2025-04-05 19:47:30,910 - INFO - Resumen de resultados guardado en: C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\results\2025-03-30_FG\identificacion_patogeno\blast_results_summary.csv
2025-04-05 19:47:30,913 - INFO - 
Mejores hits encontrados:
             evalue  bit_score  percent_identity  query_id  \
organism     

#MULTIPLE ALIGNMENT

In [43]:
import requests
from pathlib import Path
import logging
import time
import os

RESULTS_PATH = Path(r"C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\results\2025-03-30_FG\identificacion_patogeno")

def setup_logging():
    """Configura el sistema de logging"""
    RESULTS_PATH.mkdir(parents=True, exist_ok=True)
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(RESULTS_PATH / "clustal_remote.log"),
            logging.StreamHandler()
        ]
    )

def validate_fasta(fasta_data: str) -> bool:
    """Valida el formato básico del archivo FASTA"""
    lines = fasta_data.split('\n')
    if not lines[0].startswith('>'):
        return False
    return True

def run_remote_clustal(input_fasta: Path) -> Path:
    """
    Ejecuta Clustal Omega remoto usando la API de EMBL-EBI.
    
    Args:
        input_fasta: Archivo FASTA con secuencias a alinear.
        
    Returns:
        Ruta al archivo de alineamiento en formato CLUSTAL.
    """
    output_aln = RESULTS_PATH / "remote_alignment.clustal"
    
    # Leer y validar el archivo FASTA
    with open(input_fasta, 'r') as f:
        fasta_data = f.read().strip()
    
    if not validate_fasta(fasta_data):
        raise ValueError("Formato FASTA inválido. El archivo debe comenzar con un encabezado '>'")
    
    # Verificar tamaño del archivo (límite ~1MB)
    file_size = os.path.getsize(input_fasta) / 1024  # KB
    if file_size > 1024:
        raise ValueError(f"El archivo es demasiado grande ({file_size:.2f} KB). Límite: 1024 KB")
    
    # Configurar la solicitud a la API
    url = "https://www.ebi.ac.uk/Tools/services/rest/clustalo/run"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "email": "fgarciao2@correo.uss.cl",  # Obligatorio
        "sequence": fasta_data,
        "outfmt": "clustal",
        "stype": "dna",  # Especifica que son secuencias de ADN
        "dealign": "true"  # Para realinear secuencias previamente alineadas
    }
    
    logging.info("Enviando secuencias a Clustal Omega remoto...")
    
    try:
        # Paso 1: Enviar el trabajo
        response = requests.post(url, headers=headers, data=data)
        
        # Verificar respuesta detallada
        if response.status_code != 200:
            error_msg = f"Error {response.status_code}: {response.text}"
            if response.status_code == 400:
                error_msg += "\nPosibles causas:\n"
                error_msg += "- Formato FASTA incorrecto\n"
                error_msg += "- Secuencias demasiado largas\n"
                error_msg += "- Caracteres inválidos en las secuencias"
            raise RuntimeError(error_msg)
            
        job_id = response.text
        logging.info(f"Job ID recibido: {job_id}")
        
        # Paso 2: Verificar estado del trabajo
        status_url = f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/status/{job_id}"
        start_time = time.time()
        timeout = 600  # 10 minutos de timeout
        
        while True:
            if time.time() - start_time > timeout:
                raise TimeoutError("Tiempo de espera excedido para el alineamiento")
                
            status_response = requests.get(status_url)
            status = status_response.text
            logging.info(f"Estado actual: {status}")
            
            if status == "FINISHED":
                break
            elif status in ("FAILED", "ERROR"):
                raise RuntimeError(f"Error en Clustal Omega: {status}")
                
            time.sleep(10)  # Esperar 10 segundos entre verificaciones
        
        # Paso 3: Obtener resultados
        result_url = f"https://www.ebi.ac.uk/Tools/services/rest/clustalo/result/{job_id}/clustal"
        result_response = requests.get(result_url)
        result_response.raise_for_status()
        
        # Guardar el alineamiento
        with open(output_aln, 'w') as f:
            f.write(result_response.text)
        
        logging.info(f"Alineamiento guardado en: {output_aln}")
        return output_aln
        
    except requests.exceptions.RequestException as e:
        logging.error(f"Error en la conexión: {str(e)}")
        if hasattr(e, 'response') and e.response is not None:
            logging.error(f"Respuesta del servidor: {e.response.text}")
        raise

if __name__ == "__main__":
    setup_logging()
    try:
        # Corregir la ruta del archivo de entrada
        input_seqs = RESULTS_PATH / "cleaned_sequences.fasta"
        
        if not input_seqs.exists():
            raise FileNotFoundError(f"No se encontró el archivo FASTA: {input_seqs}")
        
        logging.info(f"Procesando archivo: {input_seqs}")
        aln_file = run_remote_clustal(input_seqs)
        logging.info("¡Alineamiento completado con éxito!")
        
    except Exception as e:
        logging.error(f"Error en alineamiento remoto: {str(e)}", exc_info=True)

2025-04-05 20:20:04,360 - INFO - Procesando archivo: C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\results\2025-03-30_FG\identificacion_patogeno\cleaned_sequences.fasta
2025-04-05 20:20:04,362 - INFO - Enviando secuencias a Clustal Omega remoto...
2025-04-05 20:20:05,622 - ERROR - Error en alineamiento remoto: Error 400: <?xml version='1.0' encoding='UTF-8'?>
<error>
 <description>Invalid parameters: 
Sequence -> Duplicated ID: unknown. Two sequences cannot share the same identifier</description>
</error>
Posibles causas:
- Formato FASTA incorrecto
- Secuencias demasiado largas
- Caracteres inválidos en las secuencias
Traceback (most recent call last):
  File "C:\Users\fgarc\AppData\Local\Temp\ipykernel_18448\2831071403.py", line 130, in <module>
    aln_file = run_remote_clustal(input_seqs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\fgarc\AppData\Local\Temp\ipykernel_18448\2831071403.py", line 77, in run_remote_clustal
   

#VISUALIZACION

In [ ]:
# 03_visualize.py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import logging
import numpy as np
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio.Phylo import draw
from Bio import AlignIO, Phylo
import pylab

# Configuración
RESULTS_PATH = Path(r"C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\results\2025-03-30_FG\identificacion_patogeno")
PLOT_STYLE = "seaborn-v0_8-poster"
COLOR_PALETTE = "viridis"

def setup_logging():
    RESULTS_PATH.mkdir(parents=True, exist_ok=True)
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(RESULTS_PATH / "visualization.log"),
            logging.StreamHandler()
        ]
    )

def plot_blast_results():
    """Genera múltiples visualizaciones de los resultados BLAST"""
    # Configurar estilo
    plt.style.use(PLOT_STYLE)
    sns.set_palette(COLOR_PALETTE)
    
    # Cargar datos
    csv_path = RESULTS_PATH / "blast_results_detailed.csv"
    df = pd.read_csv(csv_path)
    
    if df.empty:
        logging.warning("No hay datos para visualizar")
        return []
    
    # Filtrar los mejores hits por query
    top_hits = df.loc[df.groupby("query_id")["evalue"].idxmin()]
    
    # 1. Gráfico de los mejores hits por E-value
    plt.figure(figsize=(12, 8))
    top_organisms = top_hits.nsmallest(20, "evalue")
    # Calcular -log10(evalue) para mejor visualización
    top_organisms["-log_evalue"] = -np.log10(top_organisms["evalue"].astype(float) + 1e-300)  # Evitar log(0)
    sns.barplot(
        data=top_organisms,
        y="organism",
        x="-log_evalue",
        estimator=np.median,
        errorbar=None
    )
    plt.title("Top 20 Organismos por E-value (mejores hits)")
    plt.xlabel("-log10(E-value)")
    plt.ylabel("Organismo")
    plt.tight_layout()
    plot1 = RESULTS_PATH / "blast_top_organisms.png"
    plt.savefig(plot1, dpi=300, bbox_inches='tight')
    plt.close()
    
    # 2. Heatmap de identidad de secuencia
    plt.figure(figsize=(12, 8))
    pivot_data = top_hits.pivot_table(
        index="organism",
        columns="query_id",
        values="percent_identity",
        aggfunc="mean"
    ).fillna(0)
    sns.heatmap(
        pivot_data,
        cmap="YlOrRd",
        annot=True,
        fmt=".1f",
        linewidths=.5
    )
    plt.title("Porcentaje de Identidad por Organismo y Query")
    plt.tight_layout()
    plot2 = RESULTS_PATH / "blast_identity_heatmap.png"
    plt.savefig(plot2, dpi=300, bbox_inches='tight')
    plt.close()
    
    # 3. Gráfico de distribución de E-values
    plt.figure(figsize=(10, 6))
    sns.histplot(
        data=top_hits,
        x="evalue",
        bins=50,
        log_scale=True
    )
    plt.title("Distribución de E-values de los mejores hits")
    plt.xlabel("E-value (log scale)")
    plt.ylabel("Frecuencia")
    plt.tight_layout()
    plot3 = RESULTS_PATH / "blast_evalue_distribution.png"
    plt.savefig(plot3, dpi=300, bbox_inches='tight')
    plt.close()
    
    logging.info(f"Gráficos guardados en:\n- {plot1}\n- {plot2}\n- {plot3}")
    return [plot1, plot2, plot3]

def generate_phylogenetic_tree():
    """Genera un árbol filogenético basado en los alineamientos"""
    try:
        # Esto requeriría un archivo de alineamiento múltiple (ej. de CLUSTAL)
        aln_file = RESULTS_PATH / "alignment.clustal"
        
        if not aln_file.exists():
            logging.warning("Archivo de alineamiento no encontrado. Ejecuta CLUSTAL primero.")
            return None
            
        alignment = AlignIO.read(aln_file, "clustal")
        
        # Calcular matriz de distancia
        calculator = DistanceTreeConstructor()
        dm = calculator.get_distance(alignment)
        
        # Construir árbol (método UPGMA)
        tree = calculator.upgma(dm)
        
        # Dibujar árbol
        plt.figure(figsize=(15, 10))
        Phylo.draw(tree, do_show=False)
        plt.title("Árbol Filogenético basado en alineamiento múltiple")
        
        plot_path = RESULTS_PATH / "phylogenetic_tree.png"
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        plt.close()
        
        logging.info(f"Árbol filogenético guardado en: {plot_path}")
        return plot_path
        
    except Exception as e:
        logging.error(f"Error al generar árbol filogenético: {e}")
        return None

if __name__ == "__main__":
    setup_logging()
    try:
        plots = plot_blast_results()
        tree_plot = generate_phylogenetic_tree()
        
        if tree_plot:
            plots.append(tree_plot)
            
    except Exception as e:
        logging.error(f"Error en visualización: {e}", exc_info=True)

#ANNOTATION

In [ ]:
from Bio import Entrez, SeqIO
from pathlib import Path
import logging
import pandas as pd
import time

# Configuración
Entrez.email = "tu.email@institucion.edu"  # Obligatorio para usar NCBI
RESULTS_PATH = Path(r"C:\Users\fgarc\OneDrive\Escritorio\Doctorado\Ramos\1° Semestre\Troncal\proyecto_troncal2\results\2025-03-30_FG\identificacion_patogeno")
NCBI_DB = "nucleotide"
MAX_RETRIES = 3
RETRY_DELAY = 5  # segundos

def setup_logging():
    """Configura el sistema de logging"""
    RESULTS_PATH.mkdir(parents=True, exist_ok=True)
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(RESULTS_PATH / "annotation.log"),
            logging.StreamHandler()
        ]
    )

def fetch_genome(accession: str) -> Path:
    """
    Descarga un genoma completo desde NCBI
    
    Args:
        accession: Número de acceso del genoma
        
    Returns:
        Ruta al archivo FASTA descargado
    """
    output_file = RESULTS_PATH / f"{accession}.fasta"
    
    if output_file.exists():
        logging.info(f"Archivo ya existe: {output_file}")
        return output_file
    
    for attempt in range(MAX_RETRIES):
        try:
            logging.info(f"Descargando genoma {accession} (intento {attempt+1})...")
            
            # Obtener el genoma
            handle = Entrez.efetch(
                db=NCBI_DB,
                id=accession,
                rettype="fasta",
                retmode="text"
            )
            
            # Guardar en archivo
            with open(output_file, "w") as f:
                f.write(handle.read())
            
            handle.close()
            logging.info(f"Genoma descargado en: {output_file}")
            return output_file
            
        except Exception as e:
            logging.warning(f"Error en intento {attempt+1}: {e}")
            if attempt < MAX_RETRIES - 1:
                time.sleep(RETRY_DELAY)
    
    raise RuntimeError(f"No se pudo descargar el genoma {accession} después de {MAX_RETRIES} intentos")

def annotate_genome(fasta_file: Path) -> pd.DataFrame:
    """
    Realiza una anotación básica del genoma usando NCBI
    
    Args:
        fasta_file: Archivo FASTA con el genoma
        
    Returns:
        DataFrame con las características anotadas
    """
    try:
        # Leer el genoma
        record = next(SeqIO.parse(fasta_file, "fasta"))
        
        # Obtener anotación desde NCBI
        handle = Entrez.efetch(
            db=NCBI_DB,
            id=record.id,
            rettype="gb",
            retmode="text"
        )
        
        gb_record = SeqIO.read(handle, "genbank")
        handle.close()
        
        # Extraer características
        features = []
        for feature in gb_record.features:
            if feature.type not in ["source", "gene"]:
                continue
                
            qualifiers = feature.qualifiers
            start = feature.location.start
            end = feature.location.end
            strand = feature.location.strand
            
            features.append({
                "type": feature.type,
                "start": start,
                "end": end,
                "strand": strand,
                "product": qualifiers.get("product", [""])[0],
                "gene": qualifiers.get("gene", [""])[0],
                "protein_id": qualifiers.get("protein_id", [""])[0],
                "note": qualifiers.get("note", [""])[0]
            })
        
        return pd.DataFrame(features)
        
    except Exception as e:
        logging.error(f"Error en anotación: {e}")
        raise

if __name__ == "__main__":
    setup_logging()
    try:
        # Obtener el mejor hit de BLAST
        blast_summary = pd.read_csv(RESULTS_PATH / "blast_results_summary.csv")
        best_hit = blast_summary.iloc[0]
        
        # Extraer número de acceso (simplificado)
        accession = best_hit["accessions"].split(",")[0].strip()
        logging.info(f"Mejor hit encontrado: {accession}")
        
        # Descargar genoma
        genome_file = fetch_genome(accession)
        
        # Anotar genoma
        annotations = annotate_genome(genome_file)
        
        # Guardar anotaciones
        anno_path = RESULTS_PATH / f"{accession}_annotations.csv"
        annotations.to_csv(anno_path, index=False)
        logging.info(f"Anotaciones guardadas en: {anno_path}")
        
    except Exception as e:
        logging.error(f"Error en anotación genómica: {e}", exc_info=True)